In [21]:
import sys
import os

def remove_pngtag(lines):

    ret_lines = []
    
    find_pngtag = False
    
    for ln in lines:
        
        if ln.startswith('<div class="output_png output_subarea ">'):
            find_pngtag = True
            continue
        
        if find_pngtag == True:
            if ln.startswith('</div>'):
                find_pngtag = False
            continue
        
        ret_lines.append(ln)

    return ret_lines

def remove_prompttag(lines):

    ret_lines = []
    
    for ln in lines:
        
        if ln.startswith('<div class="prompt input_prompt">In&nbsp;'):
            ln = '<div class="prompt input_prompt"></div>'
        
        ret_lines.append(ln)

    return ret_lines

def remove_blanklines(lines):
    
    ret_lines = []
    
    blank_flag = False
    
    for ln in lines:
        
        if len(ln) == 1:
            blank_flag = True
            continue
        
        if blank_flag:
            ret_lines.append('\n')
            blank_flag = False
                    
        ret_lines.append(ln)

    return ret_lines

def remove_epochlines(lines):

    ret_lines = []
    
    epoch_flag = False
    watch_dog = 2
    
    epoch_lines = []
    
    for ln in lines:
        
        if ln.startswith('Epoch'):
            epoch_flag = True
            watch_dog = 2
            epoch_lines.append(ln)
            continue
        else:
            watch_dog -= 1

        if epoch_flag == True and watch_dog < 0:
            epoch_flag = False
            watch_dog = 2
            
            if len(epoch_lines) > 20 :
                new_epoch_lines = epoch_lines[:6]
                new_epoch_lines.append('...\n')
                new_epoch_lines.extend(epoch_lines[-7:])
            else:
                new_epoch_lines = epoch_lines
                
            ret_lines.extend(new_epoch_lines)
            
            epoch_lines = []

        if epoch_flag :
            epoch_lines.append(ln)

            continue

        ret_lines.append(ln)

    return ret_lines
        
def remove_links(lines):

    ret_lines = []
    
    find_tag = False
    find_tag2 = False
    count_tag = 0
    
    for ln in lines:
        
        if ln.startswith('<h3 id="&#50836;&#50557;">&#50836;&#50557;'):
            find_tag = True
            
        if find_tag == True:
            if ln.startswith('<div class="cell border-box-sizing text_cell rendered">'):
                find_tag2 = True
            
            if find_tag2 == True:
                if ln.startswith('</div>'):
                    count_tag += 1
                
                if count_tag == 3:
                    find_tag = False                    
                    find_tag2 = False
                continue
                
        ret_lines.append(ln)

    return ret_lines

    
    
    return ret_lines

def refine_md(src, dst):
    
    fr = open(src)
    lines = fr.readlines()
    fr.close()
    
    lines = remove_epochlines(lines)
    lines = remove_pngtag(lines)
    lines = remove_prompttag(lines) 
    lines = remove_links(lines)     
    lines = remove_blanklines(lines)    
    
    fw = open(dst, 'w')
    fw.writelines(lines)
    fw.close()
    
    print('src :' + src)
#    print('dst :' + dst)

    os.remove(src)
    
    return

In [22]:
import os

target_dir = '/Users/tykimos/Downloads'
release_dir = '/Users/tykimos/Projects/Keras/_writing/book'

target_filepath_list = []
target_filename_list = []

for (path, dir, files) in os.walk(target_dir):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.html' and filename.startswith('2017'):            
            target_filepath_list.append(path + '/' + filename)
            target_filename_list.append(filename)

for i in range(len(target_filepath_list)):
    refine_md(target_filepath_list[i], release_dir + '/' + target_filename_list[i])

src :/Users/tykimos/Downloads/2017-8-17-Text_Input_Binary_Classification_Model_Recipe.html
